In [4]:
# things we need for NLP
import nltk
import pickle
import nltk
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
import json
import random
import numpy as np
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random


# restore all of our data structures
data = pickle.load( open( "training_data.parquet", "rb" ) )
# words = data['words']
# classes = data['classes']
# train_x = data['train_x']
# train_y = data['train_y']
# print(classes)
# print(train_x[0])
# print(train_y[0])


words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [5]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))
print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique lemmatized words", words)

97 documents
12 classes ['entertainment_movies', 'entertainment_music', 'flight_assistant', 'goodbye', 'greeting', 'options', 'order_dessert', 'order_drinks', 'order_food', 'tellmejoke', 'thanks', 'weather_search']
119 unique lemmatized words ["'s", ',', 'a', 'again', 'album', 'all', 'am', 'amazing', 'anybody', 'anyone', 'are', 'assistant', 'at', 'available', 'beverage', 'bored', 'bot', 'boy', 'bro', 'bunch', 'bye', 'call', 'can', 'catch', 'cheer', 'cool', 'day', 'dessert', 'do', 'doubt', 'drink', 'eat', 'else', 'everybody', 'everything', 'farewell', 'flight', 'folk', 'food', 'for', 'give', 'good', 'goodbye', 'goodnight', 'great', 'have', 'hello', 'help', 'helpful', 'here', 'hey', 'hi', 'hola', 'how', 'howdy', 'i', 'in', 'is', 'it', 'item', 'joke', 'know', 'later', 'let', 'like', 'list', 'listen', 'long', 'me', 'menu', 'movie', 'much', 'music', "n't", 'navigate', 'need', 'nice', 'night', 'now', 'of', 'on', 'option', 'order', 'outside', 'play', 'please', 'possible', 'robot', 'see', 'sho

In [6]:
import cloudpickle as cp
dataprep_pipeline = cp.load(open('chatbot_data_preparation.sav', 'rb'))

In [7]:
training = dataprep_pipeline.transform(documents)
print(training[0])
train_x = list(training[:,0])
train_y = list(training[:,1])


[list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
 list([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])]


### Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons equal to number of intents to predict output intent with softmax

In [8]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
dropout=0.5
optimizer=sgd

# def build_classifier(optimizer='adam', dropout=0.5):
classifier = Sequential()
classifier.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
classifier.add(Dropout(dropout))
classifier.add(Dense(64, activation='relu'))
classifier.add(Dropout(dropout))
classifier.add(Dense(len(train_y[0]), activation='softmax'))
classifier.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
# return classifier


### Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model

In [9]:
from sklearn.pipeline import Pipeline
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import GridSearchCV

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
hist = classifier.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=5, verbose=1)

#  classifier = KerasClassifier(build_fn= build_classifier ,optimizer=sgd, epochs=100, batch_size=5, verbose=1)
# pipeline_keras = Pipeline([('clf', classifier)])

# hist = pipeline_keras.fit(np.array(train_x), np.array(train_y))

# param_grid = {
#     'clf__optimizer':[sgd,'adam'],
#     'clf__epochs':[100],
#     'clf__dropout':[0.5,0.4],
# }
# grid_search = GridSearchCV(estimator = pipeline,
#                        param_grid = param_grid,
#                        scoring = 'accuracy',
#                        cv=2)


# grid_search.fit(np.array(train_x), np.array(train_y))



Epoch 1/100
20/20 [==============================] - 0s 1ms/step - loss: 2.4473 - accuracy: 0.1649
Epoch 2/100
20/20 [==============================] - 0s 1ms/step - loss: 2.3181 - accuracy: 0.1856
Epoch 3/100
20/20 [==============================] - 0s 1ms/step - loss: 2.2492 - accuracy: 0.2371
Epoch 4/100
20/20 [==============================] - 0s 1ms/step - loss: 2.0696 - accuracy: 0.2887
Epoch 5/100
20/20 [==============================] - 0s 1ms/step - loss: 1.9437 - accuracy: 0.3299
Epoch 6/100
20/20 [==============================] - 0s 1ms/step - loss: 1.8271 - accuracy: 0.3608
Epoch 7/100
20/20 [==============================] - 0s 1ms/step - loss: 1.6976 - accuracy: 0.4639
Epoch 8/100
20/20 [==============================] - 0s 1ms/step - loss: 1.5427 - accuracy: 0.5155
Epoch 9/100
20/20 [==============================] - 0s 1ms/step - loss: 1.3336 - accuracy: 0.6082
Epoch 10/100
20/20 [==============================] - 0s 1ms/step - loss: 1.1747 - accuracy: 0.6804
Epoch 11/

Epoch 83/100
20/20 [==============================] - 0s 1ms/step - loss: 0.0844 - accuracy: 0.9794
Epoch 84/100
20/20 [==============================] - 0s 1ms/step - loss: 0.0453 - accuracy: 0.9897
Epoch 85/100
20/20 [==============================] - 0s 1ms/step - loss: 0.0470 - accuracy: 0.9897
Epoch 86/100
20/20 [==============================] - 0s 1ms/step - loss: 0.0410 - accuracy: 0.9897
Epoch 87/100
20/20 [==============================] - 0s 1ms/step - loss: 0.0642 - accuracy: 0.9691
Epoch 88/100
20/20 [==============================] - 0s 986us/step - loss: 0.0570 - accuracy: 0.9897
Epoch 89/100
20/20 [==============================] - 0s 1ms/step - loss: 0.0658 - accuracy: 0.9897
Epoch 90/100
20/20 [==============================] - 0s 1ms/step - loss: 0.0410 - accuracy: 1.0000
Epoch 91/100
20/20 [==============================] - 0s 1ms/step - loss: 0.0921 - accuracy: 0.9691
Epoch 92/100
20/20 [==============================] - 0s 2ms/step - loss: 0.0280 - accuracy: 1.000

### saving the model 

In [10]:
hist2 = classifier.fit(np.array(train_x), np.array(train_y))


4/4 [==============================] - 0s 4ms/step - loss: 0.0320 - accuracy: 0.9897


In [16]:
import os
import cloudpickle as cp

default_filename = "chatbot_model.h5"
filename = os.getenv("S2I_PIPELINE_STAGE_SAVE_FILE", default_filename)

# cp.dump(classifier, open(default_filename, "wb"))
# cp.dump(classifier, open(filename, "wb"))
classifier.save(default_filename)
classifier.save(filename)